## Import necessary packages (run once upon startup)

In [2]:
from __future__ import division 
import os
import openpyxl
import pandas as pd
import numpy as np
import math
import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.style.use("ggplot")
%matplotlib inline


from skimage.transform import resize
from skimage.morphology import skeletonize
from scipy.signal import resample, savgol_filter, butter, filtfilt
from PIL import Image, ImageDraw
import cv2
from cv2 import EVENT_LBUTTONDOWN

import tensorflow as tf

from keras import backend as K
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# Custom function definitions (run once upon startup)

## Import image & prepare directory

In [2]:
# Get list of Files in directory
def get_list_of_files(pathname):
    return glob.glob(pathname)

# Import image and reshape to desired size
def import_reshape_image(path_to_image):
    
    # Define the image to analyse here and load it
    image_add = path_to_image

    filename = os.path.splitext(os.path.basename(image_add))[0]
    # img = load_img(image_add, color_mode='grayscale')
    img = cv2.imread(path_to_image, 0)
    print("Loaded image at " + path_to_image)
    nonflipped_img = img
    img_copy = img
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(20,20))
    img = clahe.apply(img)
    img = img_to_array(img)
    h = img.shape[0]
    w = img.shape[1]
    img = np.reshape(img,[-1, h, w,1])
    img = resize(img, (1, 256, 256, 1), mode = 'constant', preserve_range = True)
    img = img/255.0
    img2 = img
    return filename, img, img_copy, nonflipped_img, h, w;

def get_flip_flags_list(pathname):
    flipFlags = []
    file = open(pathname, 'r')
    for line in file:
        for digit in line:
            if digit.isdigit():
                flipFlags.append(digit)
    return flipFlags
        

In [14]:
img = load_img("C:/Users/Paul/Desktop/img_1.tiff")
img1 = load_img("C:/Users/Paul/Desktop/img_2.tiff")
img_n1 =(img_to_array(img) / 255.0)
img_n2 =(img_to_array(img1) / 255.0)

m1 = np.mean(img)
m2 = np.mean(img1)
mn1 = np.mean(img_n1)
mn2 = np.mean(img_n2)

div1 = m2 / m1
div2 = mn2 / mn1

print(m1, m2, mn1, mn2, div1, div2)



29.779856770833334 16.573812362938597 0.11678381 0.064995356 0.556544394772615 0.55654424


## Image scaling

In [3]:
# Function to detect mouse clicks for the purpose of image calibration
def mclick(event, x, y, flags, param):
    
    global mlocs 
    # if the left mouse button was clicked, record the (x, y) coordinates
    if event == cv2.EVENT_LBUTTONDOWN:
        mlocs.append(y)

def calibrate_distance_manually(nonflipped_img, spacing, depth):
    # Calibrate the analysis by clicking on 2 points in the image, followed by the 'q' key. These two points should be 1cm apart
    # Alternatively, change the spacing setting below
    # NOTE: Here we assume that the points are spaced apart in the y/vertical direction of the image
    img2 = np.uint8(nonflipped_img)

    # display the image and wait for a keypress
    cv2.imshow("image", img2)
    cv2.setMouseCallback("image", mclick)
    key = cv2.waitKey(0)
 
    # if the 'q' key is pressed, break from the loop
    if key == ord("q"):
        cv2.destroyAllWindows()

    calib_dist = np.abs(mlocs[0] - mlocs[1])
    scalingline_length = depth * calib_dist
    print(str(spacing) + ' mm corresponds to ' + str(calib_dist) + ' pixels')
    return scalingline_length

# Function to calibrate US images with intermittend scaling bars 
def calibrate_distance_static(nonflipped_img, spacing, depth): 
    #calibrate analysis according to scale at the right border of the image
    img2 = np.uint8(nonflipped_img)
    imgscale = img2[70:,1100:1115]#cut out the rightmost part of the picture (the depth scale) by pixels

    #search for rows with white pixels, calculate median of distance (index difference) between those rows
    calib_dist = np.median(np.diff(np.argwhere(imgscale.sum(axis=1)>200),axis=0))
    scalingline_length = depth * calib_dist
    print(str(spacing) + ' mm corresponds to ' + str(calib_dist) + ' pixels')
    return scalingline_length

## Optional, just for plotting 
def plot_image(image):
    img = image
    fig, (ax1)= plt.subplots(1, 1, figsize = (15, 15))
    ax1.imshow(img, cmap = "gray")
    ax1.grid(False)
    plt.savefig("Ridge_test_1.tif")
    
def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    #channel_count = img.shape[2]
    match_mask_color = 255
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_the_lines(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8) # Creating empty image to draw lines on 

    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(blank_image, (x1,y1), (x2,y2), (0, 255, 0), thickness=1)

    img = cv2.addWeighted(img, 0.8, blank_image, 1, 0.0) # Overlay image with lines on original images (only needed for plotting)
    return img

# Function to calibrate US images with scaling line
def calibrate_distance_efov(path_to_image):

    image = cv2.imread(path_to_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Transform BGR Image to RGB
    height = image.shape[0]
    width = image.shape[1]
    # Define ROI with scaling lines
    region_of_interest_vertices = [ 
        (150, height),
        (150, 80),
        (1100, 80),
        (1100, height)
    ]
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) # Transform RGB to greyscale for edge detection
    canny_image = cv2.Canny(gray_image, 400, 600) # Edge detecition
    cropped_image = region_of_interest(canny_image,
                    np.array([region_of_interest_vertices], np.int32),) 
    
    # For RF
    if muscle == "RF": 
        lines = cv2.HoughLinesP(cropped_image,
                                rho=1,
                                theta=np.pi/180,
                                threshold=50,
                                lines=np.array([]),
                                minLineLength=400,
                                maxLineGap=1)
        print(lines)
        image_with_lines = draw_the_lines(image, lines)

    # For VL 
    if muscle == "VL": 
        lines = cv2.HoughLinesP(cropped_image,
                                rho=1, # Distance of pixels in accumulator
                                theta=np.pi/180, # Angle resolution in accumulator
                                threshold=50, # Only only those lines rutrnes that have higher vote value
                                lines=np.array([]),
                                minLineLength=200,
                                maxLineGap=3) # Gap between lines
        image_with_lines = draw_the_lines(image, lines)

    # Calculate length of the scaling line
    scalingline = lines[0][0]
    point1 = [scalingline[0], scalingline[1]]
    point2 = [scalingline[2], scalingline[3]]
    scalingline_length = math.sqrt(((point1[0]-point2[0])**2)
                                   +((point1[1]-point2[1])**2))
    #plot_image(image_with_lines)
    return scalingline_length
    



## IoU for modelimport

In [ ]:

# Intersection over union (IoU), a measure of labelling accuracy (sometimes also called Jaccard score)
def IoU(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    union = K.sum(y_true,-1) + K.sum(y_pred,-1) - intersection
    iou = (intersection + smooth) / ( union + smooth)
    return iou


## Model prediction

In [ ]:
def do_predictions(img, h, w, filename):
    # Get NN predictions for the image

    apo_threshold = 0.5 # Set threshold with minimal confidence to make binary 
    pred_apo = model_apo.predict(img)
    pred_apo_t = (pred_apo > apo_threshold).astype(np.uint8) # SET APO THRESHOLD -> makes binary, unisned integer between 0 and 255 = np.uint8
    
    img = resize(img, (1, h, w, 1))
    img = np.reshape(img, (h, w))
    pred_apo = resize(pred_apo, (1, h, w,1))
    pred_apo = np.reshape(pred_apo, (h, w))
    pred_apo_t = resize(pred_apo_t, (1, h, w,1))
    pred_apo_t = np.reshape(pred_apo_t, (h, w))

    # Uncomment these lines if you want to see the initial predictions
    fig = plt.figure(figsize=(20,20))
    ax1 = fig.add_subplot(1,2,1) #Fist is n rows in grid, second n columns, third is position
    ax1.imshow(img.squeeze(),cmap='gray')
    ax1.grid(False)
    ax1.set_title('Original image')
    ax2 = fig.add_subplot(1,2,2)
    ax2.imshow(pred_apo_t.squeeze(),cmap="gray")
    ax2.grid(False)
    ax2.set_title('Aponeuroses')
    #plt.savefig("./analyzed_images" + filename)
    
    return pred_apo_t, fig

## Area calculation and saving

In [ ]:
# Calculate area
def calc_area(depth, scalingline_length, img):

    pix_per_cm = scalingline_length / depth
    pred_muscle_area = cv2.countNonZero(img) / pix_per_cm**2
    #print(pred_muscle_area)
    return pred_muscle_area
    
# Save results
def compile_save_results(rootpath, imagepath, filename, muscle, area):
    
    excelpath = rootpath + '/Results.xlsx'
    if os.path.exists(excelpath):
        with pd.ExcelWriter(excelpath, mode='a') as writer:
            data = pd.DataFrame({'Image_ID': filename, 'Muscle': muscle, 'Area_cm²': area}, index = [0])
            data.to_excel(writer)
    else:
        with pd.ExcelWriter(excelpath, mode='w') as writer:
            data = pd.DataFrame({'Image_ID': filename, 'Muscle': muscle, 'Area_cm²': area}, index = [0])
            data.to_excel(writer)
    
# Function to calculate for whole batch
def calculate_batch_efov(rootpath):
    list_of_files = glob.glob(rootpath + '/**/*.tif', recursive=True)
    
    with PdfPages(rootpath + '/Analyzed_images.pdf') as pdf:  
            
        for imagepath in list_of_files:

            filename, img, img_copy, nonflipped_img, h, w = import_reshape_image(imagepath) 
            scalingline_length = calibrate_distance_efov(imagepath)
                   
            img, fig = do_predictions(img, h, w, filename)
            area = calc_area(depth, scalingline_length, img)
            compile_save_results(rootpath, imagepath, filename, muscle, area)
            pdf.savefig(fig)
            plt.close(fig)
    

# Function to calculate for whole batch
def calculate_batch(rootpath, flip_file_path):
    list_of_files = glob.glob(rootpath + '/**/*.tif', recursive=True)
    flip_flags = get_flip_flags_list(flip_file_path)
    
    with PdfPages(rootpath + '/Analyzed_images.pdf') as pdf:
        
        if(len(list_of_files) == len(flip_flags)):
            
            for imagepath in list_of_files:
                
                flip = flip_flags.pop(0)
                filename, img, img_copy, nonflipped_img, h, w = import_reshape_image(imagepath)
                
                if scaling == "Static": 
                    
                    scalingline_length = calibrate_distance_static(nonflipped_img, spacing, depth)
                
                else: 
                    
                    scalingline_length = calibrate_distance_manually(nonflipped_img, spacing, depth)
                                       
                img, fig = do_predictions(img, h, w, filename)
                area = calc_area(depth, scalingline_length, img)
                compile_save_results(rootpath, imagepath, filename, muscle, area)
                pdf.savefig(fig)
                plt.close(fig)
        else:
            print("Warning: number of flipFlags doesn\'t match number of images! Calculations aborted.") 


## Global Variables

In [ ]:
model_apo = load_model("C:/Users/Paul/Desktop/Test_image/model/model.h5", custom_objects={'IoU': IoU})
depth = 6
muscle = "RF" # VL / GM
scaling = "EFOV" # Static / Manual
spacing = 5 
mlocs = []

# MAIN

In [ ]:
if scaling == "EFOV": 
    calculate_batch_efov("C:/Users/paul/Desktop/Test_image")
else: 
    calculate_batch("C:/Users/paul/Desktop/Test_image",
                        "C:/Users/paul/Desktop/Test_image/Flip.txt")